In [1]:
from datasets import load_dataset
from datasets import Dataset, Image
from glob import glob
from tqdm import tqdm
import linecache
import json
import mp
import os

In [2]:
text_file = open('locations.txt', 'r')
lines = text_file.readlines()
keyword_list = list(dict.fromkeys(lines))

In [3]:
results = []
for no, keyword in enumerate(keyword_list):
    filename = f'data/generated-location/{no}.jsonl'
    results.append((keyword, filename))

In [4]:
from tqdm import tqdm

def loop(files):
    files, _ = files
    results = []
    for file in tqdm(files):
        keyword, file = file
        if not os.path.exists(file):
            continue
        keyword = keyword.strip()
        index = file.split('/')[-1].replace('.jsonl', '')
        with open(file) as fopen:
            i = 0
            for l in fopen:
                try:
                    l = json.loads(l)
                except:
                    i += 1
                    continue
                filename = os.path.join('image-location', f'{index}-{i}.jpeg')
                if not os.path.exists(filename):
                    i += 1
                    continue
                    
                l.pop('image_base64', None)
                    
                l['filename'] = filename
                l['image'] = filename
                l['keyword'] = keyword
                    
                results.append(l)
                
                i += 1
                
    return results

In [5]:
r = mp.multiprocessing(results, loop, cores = 30, returned = True)

 36%|███▌      | 651/1806 [03:20<04:04,  4.71it/s]]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
len(r)

In [7]:
dataset = Dataset.from_list(r)
dataset = dataset.cast_column("image", Image())
dataset

In [8]:
dataset[0]

In [9]:
dataset[0]['image']

In [10]:
dataset[-1]

In [11]:
dataset[-1]['image']

In [12]:
dataset[100001]

In [13]:
dataset[100001]['image']

In [14]:
# import time

# while True:
#     try:
#         dataset.push_to_hub(repo_id = 'malaysia-ai/crawl-google-image-malaysia-location')
#     except Exception as e:
#         print(e)
#         pass
    
#     time.sleep(60 * 60)

In [15]:
from huggingface_hub import HfApi
from io import BytesIO
from datasets.table import embed_table_storage

api = HfApi()

In [23]:


def shards_with_embedded_external_files(shard):
    format = shard.format
    shard = shard.with_format("arrow")
    shard = shard.map(
        embed_table_storage,
        batched=True,
        batch_size=1000,
        keep_in_memory=True,
    )
    shard = shard.with_format(**format)
    return shard

# shards = shards_with_embedded_external_files(shards)

In [17]:
!mkdir done-shard
# !rm -rf done-shard/*

In [25]:
import time

while True:
    try:
        num_shards = 1000
        shards = (dataset.shard(num_shards=num_shards, index=i, contiguous=True) for i in range(num_shards))
        for no, shard in tqdm(enumerate(shards)):
            checkpoint = os.path.join('done-shard', f'{no}')
            if os.path.exists(checkpoint):
                continue

            shard = shards_with_embedded_external_files(shard)

            shard_path_in_repo = f"data/train-{no:05d}-of-{num_shards:05d}.parquet"
            buffer = BytesIO()
            shard.to_parquet(buffer)

            api.upload_file(
                path_or_fileobj=buffer,
                path_in_repo=shard_path_in_repo,
                repo_id='malaysia-ai/crawl-google-image-malaysia-location',
                repo_type='dataset',
            )

            with open(checkpoint, 'w') as fopen:
                fopen.write('done')
        break
    except Exception as e:
        print(e)
        time.sleep(60 * 60)